# Adding Features To Labelled and Unlabelled Data

## Method 1

In [10]:
# Imports
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

In [5]:
# File Retrieval 
unlabelled = pd.read_csv('../data/data/unlabelled_data.csv')
labelled = pd.read_csv('../data/data/labelled_data.csv')

print("Unlabelled:")
unlabelled.head()

Unlabelled:


,place_id,review_id,name_person,reviewer_id,rating_person,review_text,published_ago,published_at_date,response_from_owner_text,response_from_owner_ago,...,website,phone,can_claim,owner_name,main_category,categories,workday_timing,closed_on,address,review_keywords
0,ChIJU_Rb8VIZ2jER1wWC4e0_hXw,Ci9DQUlRQUNvZENodHljRjlvT25aUFRrcERUbEpVVVV3M0...,TIFFANY,102894114421490527374,5,"The food here is really good and healthy, with...",5,2025-08-20 04:53:37,NaN,NaN,...,https://www.paperbakes.co/,6206 6631 ext. 2203,NaN,Paperbakes @ Raffles City (Owner),Restaurant,Restaurant,10:30 am-9:30 pm,Open All Days,"252 N Bridge Rd, B1-80 Raffles City, Singapore...","kale, salad, lunch"
1,ChIJU_Rb8VIZ2jER1wWC4e0_hXw,Ci9DQUlRQUNvZENodHljRjlvT2sxeFRYTTFVRVpwV0VsbF...,Joyin Chio,101329843800187896226,5,Came across this new outlet with surprisingly ...,6,2025-08-19 05:30:13,NaN,NaN,...,https://www.paperbakes.co/,6206 6631 ext. 2203,NaN,Paperbakes @ Raffles City (Owner),Restaurant,Restaurant,10:30 am-9:30 pm,Open All Days,"252 N Bridge Rd, B1-80 Raffles City, Singapore...","kale, salad, lunch"
2,ChIJU_Rb8VIZ2jER1wWC4e0_hXw,Ci9DQUlRQUNvZENodHljRjlvT2tOSFZXOHhZVVJJTlhkdl...,Cheok Sao Hoon,107241069821421169260,5,"The flavors were perfectly balanced--savory, f...",6,2025-08-19 03:33:21,NaN,NaN,...,https://www.paperbakes.co/,6206 6631 ext. 2203,NaN,Paperbakes @ Raffles City (Owner),Restaurant,Restaurant,10:30 am-9:30 pm,Open All Days,"252 N Bridge Rd, B1-80 Raffles City, Singapore...","kale, salad, lunch"
3,ChIJU_Rb8VIZ2jER1wWC4e0_hXw,Ci9DQUlRQUNvZENodHljRjlvT25SM01rWjZWeTE1ZFRaUV...,Melissa LovesIceCream,111353505751212189241,5,"Simple to order via kiosk, signature chicken w...",7,2025-08-18 04:11:57,NaN,NaN,...,https://www.paperbakes.co/,6206 6631 ext. 2203,NaN,Paperbakes @ Raffles City (Owner),Restaurant,Restaurant,10:30 am-9:30 pm,Open All Days,"252 N Bridge Rd, B1-80 Raffles City, Singapore...","kale, salad, lunch"
4,ChIJU_Rb8VIZ2jER1wWC4e0_hXw,Ci9DQUlRQUNvZENodHljRjlvT2tSVmNFaDFMVXhHY201VW...,Lawrence Neo,102591407792887898032,5,Had office lunch from paperbakes. The Whole Ch...,10,2025-08-15 09:01:47,NaN,NaN,...,https://www.paperbakes.co/,6206 6631 ext. 2203,NaN,Paperbakes @ Raffles City (Owner),Restaurant,Restaurant,10:30 am-9:30 pm,Open All Days,"252 N Bridge Rd, B1-80 Raffles City, Singapore...","kale, salad, lunch"


In [7]:
print("Labelled:")
labelled.head()

Labelled:


,place_id,review_id,name_person,reviewer_id,rating_person,review_text,published_ago,published_at_date,response_from_owner_text,response_from_owner_ago,...,phone,can_claim,owner_name,main_category,categories,workday_timing,closed_on,address,review_keywords,label
0,ChIJU_Rb8VIZ2jER1wWC4e0_hXw,Ci9DQUlRQUNvZENodHljRjlvT25aUFRrcERUbEpVVVV3M0...,TIFFANY,1.028940e+20,5,"The food here is really good and healthy, with...",5,20/8/2025 4:53,NaN,NaN,...,6206 6631 ext. 2203,NaN,Paperbakes @ Raffles City (Owner),Restaurant,Restaurant,10:30 am-9:30 pm,Open All Days,"252 N Bridge Rd, B1-80 Raffles City, Singapore...","kale, salad, lunch",relevant
1,ChIJU_Rb8VIZ2jER1wWC4e0_hXw,Ci9DQUlRQUNvZENodHljRjlvT2sxeFRYTTFVRVpwV0VsbF...,Joyin Chio,1.013300e+20,5,Came across this new outlet with surprisingly ...,6,19/8/2025 5:30,NaN,NaN,...,6206 6631 ext. 2203,NaN,Paperbakes @ Raffles City (Owner),Restaurant,Restaurant,10:30 am-9:30 pm,Open All Days,"252 N Bridge Rd, B1-80 Raffles City, Singapore...","kale, salad, lunch",relevant
2,ChIJU_Rb8VIZ2jER1wWC4e0_hXw,Ci9DQUlRQUNvZENodHljRjlvT2tOSFZXOHhZVVJJTlhkdl...,Cheok Sao Hoon,1.072410e+20,5,"The flavors were perfectly balanced--savory, f...",6,19/8/2025 3:33,NaN,NaN,...,6206 6631 ext. 2203,NaN,Paperbakes @ Raffles City (Owner),Restaurant,Restaurant,10:30 am-9:30 pm,Open All Days,"252 N Bridge Rd, B1-80 Raffles City, Singapore...","kale, salad, lunch",relevant
3,ChIJU_Rb8VIZ2jER1wWC4e0_hXw,Ci9DQUlRQUNvZENodHljRjlvT25SM01rWjZWeTE1ZFRaUV...,Melissa LovesIceCream,1.113540e+20,5,"Simple to order via kiosk, signature chicken w...",7,18/8/2025 4:11,NaN,NaN,...,6206 6631 ext. 2203,NaN,Paperbakes @ Raffles City (Owner),Restaurant,Restaurant,10:30 am-9:30 pm,Open All Days,"252 N Bridge Rd, B1-80 Raffles City, Singapore...","kale, salad, lunch",relevant
4,ChIJU_Rb8VIZ2jER1wWC4e0_hXw,Ci9DQUlRQUNvZENodHljRjlvT2tSVmNFaDFMVXhHY201VW...,Lawrence Neo,1.025910e+20,5,Had office lunch from paperbakes. The Whole Ch...,10,15/8/2025 9:01,NaN,NaN,...,6206 6631 ext. 2203,NaN,Paperbakes @ Raffles City (Owner),Restaurant,Restaurant,10:30 am-9:30 pm,Open All Days,"252 N Bridge Rd, B1-80 Raffles City, Singapore...","kale, salad, lunch",relevant


In [ ]:
# convert NaN to 0 for is_local_guide, is_spending_on_ads, can_claim
labelled['is_local_guide'] = labelled['is_local_guide'].fillna(0)
labelled['is_spending_on_ads'] = labelled['is_spending_on_ads'].fillna(0)
labelled['can_claim'] = labelled['can_claim'].fillna(0)

print(labelled.describe())

## Splitting into Train and Test

In [ ]:
y = labelled['label']
# X = labelled.drop("label", axis='columns')
X = labelled['review_text'] # only review_text because TF-IDF should only be done on review_text

# Split such that the test set is 20% of the data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Adding Features

In [ ]:
# Convert text to TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1,2), stop_words="english")
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
def add_features(revs):
    features = []
    ad_keywords = ["buy", "discount", "offer", "click here"]
    relevant_keywords = ["food", "shop", "order", "paid", "service", "restaurant", "ambience", "shop", "price", "delicious", "tasty"]

    for rev in revs: 
        review = review.lower()
        words = review.split()

        # Checking for links 
        # 0 for no links, 1 for links
        has_link = int(bool(re.search(r"http|www", review, re.IGNORECASE)))

        # Checking for review word count
        word_count = len(words)

        # Checking for number of all caps words 
        caps = 0
        for word in rev.split(): 
            if word.isupper():
                caps += 1
        
        # Checking for email
        has_email = int(bool(re.search(r'@', review)))

        # Checking for phone number 
        has_phone_number = int(bool(re.search(r'\d\d\d\d ?-?\d', review)))

        # Check if ad_keywords are present
        has_ad_words = int(bool(any(ad_kw in words for ad_kw in ad_keywords)))

        # Check if relevant_keywords are present
        has_relevant_words = int(any(rev_word in review for rev_word in relevant_keywords))

        features.append([has_link, word_count, caps, has_email, has_phone_number, has_ad_words, has_relevant_words])
    
    return np.array(features)

with_features_train = add_features(X_train)
with_features_test = add_features(X_test)

In [ ]:
# Combine TF-IDF and features
X_train_combine = hstack([X_train_tfidf, with_features_train])
X_test_combine = hstack([X_test_tfidf, with_features_test])

## Method 2

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
y2 = labelled['label']
X2 = labelled.drop("label", axis='columns')